# Utilização do Modelo Word2Vec com GenSim

## Introdução
Esta atividade envolve a pesquisa e utilização do modelo Word2Vec, utilizando a biblioteca GenSim do Python, para criar representações vetoriais de palavras a partir de um corpus escolhido. O objetivo é entender como os vetores de palavras podem capturar relações semânticas e contextuais.

## Preparação dos Dados
A preparação dos dados é um passo crucial antes de treinar o modelo Word2Vec. Este processo inclui:
- **Remoção de Stopwords**: Eliminar palavras comuns que não contribuem significativamente para o sentido do texto.
- **Lematização**: Reduzir as palavras às suas formas base, facilitando o agrupamento de variantes de uma mesma palavra.
- **Tokenização**: Dividir o texto em palavras ou frases.

## Treinamento do Modelo Word2Vec
Com os dados preparados, o próximo passo é treinar o modelo Word2Vec. Utilizamos a biblioteca GenSim para este propósito, especificando parâmetros como o tamanho do vetor, a janela de contexto, e o número mínimo de ocorrências de palavras.

## Exploração de Saídas do Word2Vec
Após treinar o modelo, podemos explorar as saídas geradas. Isso inclui:
- **Vetores de Palavras**: Cada palavra no corpus é representada por um vetor em um espaço dimensional.
- **Similaridade entre Palavras**: Podemos calcular a similaridade entre vetores de palavras, o que nos dá uma medida de quão semanticamente relacionadas as palavras estão.

## Exploração de Métodos da GenSim
A biblioteca GenSim oferece vários métodos úteis para explorar e analisar o modelo Word2Vec treinado. Isso inclui encontrar palavras similares, calcular a similaridade entre pares de palavras, e muito mais.

In [49]:
import pandas as pd
from gensim.models import Word2Vec
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import re

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/gustavohroos/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/gustavohroos/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/gustavohroos/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/gustavohroos/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [50]:
dados = pd.read_csv('Lorde.tsv', sep='\t')
dados_limpos = dados.drop(columns=['Unnamed: 0'])

In [51]:
def preparar_texto(texto):
    texto = re.sub(r'[^a-zA-Z\s]', '', texto).lower()
    
    tokens = word_tokenize(texto)
    
    tokens_limpos = [palavra for palavra in tokens if palavra not in stopwords.words('english')]
    
    lemmatizer = WordNetLemmatizer()
    
    def obter_pos_tag(word):
        tag = nltk.pos_tag([word])[0][1][0].upper()
        tag_dict = {"J": wordnet.ADJ,
                    "N": wordnet.NOUN,
                    "V": wordnet.VERB,
                    "R": wordnet.ADV}
        return tag_dict.get(tag, wordnet.NOUN)
    
    tokens_lematizados = [lemmatizer.lemmatize(w, obter_pos_tag(w)) for w in tokens_limpos]
    
    return tokens_lematizados

dados_limpos['Texto_Preparado'] = dados_limpos['Corpus'].apply(preparar_texto)
dados_limpos[['Corpus', 'Texto_Preparado']].head()


,Corpus,Texto_Preparado
0,I'm a liability I'm a liability Much for me Yo...,"[im, liability, im, liability, much, youre, li..."
1,"You asked if I was feeling it, I’m psycho high...","[ask, feel, im, psycho, high, know, wont, reme..."
2,(Go back and tell it) Please could you be tend...,"[go, back, tell, please, could, tender, sit, c..."
3,"Night, midnight, lose my mind Night, midnight,...","[night, midnight, lose, mind, night, midnight,..."
4,"In my head, I play a supercut of us All the ma...","[head, play, supercut, u, magic, give, love, l..."


In [52]:
sentencas = dados_limpos['Texto_Preparado'].tolist()

modelo_word2vec = Word2Vec(sentences=sentencas, vector_size=100, window=5, min_count=1, workers=4)

palavras_similares = modelo_word2vec.wv.most_similar('feeling', topn=10)

palavras_similares

[('though', 0.3209531307220459),
 ('different', 0.30907854437828064),
 ('psycho', 0.2959253489971161),
 ('well', 0.29396384954452515),
 ('tie', 0.286493718624115),
 ('send', 0.2776780128479004),
 ('cadillacs', 0.27569177746772766),
 ('perfect', 0.26952219009399414),
 ('else', 0.26385262608528137),
 ('grocery', 0.2627700865268707)]